In [1]:
!pip install fastapi uvicorn[standard] tensorflow scikit-learn pandas spacy nltk requests -q
# Le -q rend l'installation silencieuse
print("Dépendances FastAPI et ML installées/vérifiées.")

# Installer aussi le modèle spaCy si ce n'est pas déjà fait
!python -m spacy download en_core_web_sm -q
print("Modèle spaCy téléchargé/vérifié.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 877.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.8/459.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.6 MB/s eta 0:00:00
Dépendances FastAPI et ML installées/vérifiées.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 94.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Modèle spaCy téléchargé/vérifié.


In [2]:
# 1. Navigue vers le dossier PARENT de 'api' (toujours nécessaire)
%cd /content/drive/MyDrive/Colab Notebooks/OC/P7/

# 2. Lance Uvicorn en arrière-plan et log sa sortie
#    'nohup' : permet au processus de continuer même si tu fermes le 'terminal' (la cellule)
#    '&' : met le processus en arrière-plan immédiatement
#    '> uvicorn.log 2>&1' : redirige stdout et stderr vers un fichier uvicorn.log
print("Lancement d'Uvicorn en arrière-plan...")
!nohup uvicorn api.main:app --host 0.0.0.0 --port 8000 > uvicorn.log 2>&1 &
print("Commande lancée. Uvicorn devrait tourner en arrière-plan.")
print("Attente de quelques secondes pour le démarrage...")
import time
time.sleep(10) # Laisser le temps à Uvicorn de démarrer

# 3. Vérifier les dernières lignes du log (optionnel mais utile)
print("\nDernières lignes du log Uvicorn (uvicorn.log):")
!tail uvicorn.log

/content/drive/MyDrive/Colab Notebooks/OC/P7
Lancement d'Uvicorn en arrière-plan...
Commande lancée. Uvicorn devrait tourner en arrière-plan.
Attente de quelques secondes pour le démarrage...

Dernières lignes du log Uvicorn (uvicorn.log):
2025-05-05 07:18:44.627621: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746429524.650839     849 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746429524.657723     849 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-05 07:18:44.680847: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable 

In [3]:
# 1. Installer pyngrok (si pas déjà fait dans la session)
!pip install pyngrok -q

# 2. Importer et configurer (si token disponible)
from pyngrok import ngrok, conf
import getpass

print("Vérification/Configuration Ngrok...")
try:
    # Décommenter et remplacer par ton token si tu en as un et qu'il n'est pas dans les secrets
    # NGROK_AUTH_TOKEN = "TON_TOKEN_NGROK_ICI"
    # ngrok.set_auth_token(NGROK_AUTH_TOKEN)

    # Tenter de lire depuis les secrets Colab (préférable)
    from google.colab import userdata
    ngrok_token = userdata.get('NGROK_AUTH_TOKEN')
    if ngrok_token:
        print("Utilisation du token ngrok depuis les secrets Colab.")
        ngrok.set_auth_token(ngrok_token)
    else:
        print("Token ngrok non trouvé dans les secrets. Utilisation sans token.")
except ImportError:
     print("Secrets Colab non disponibles. Utilisation sans token ngrok.")
except Exception as e:
    print(f"Erreur lors de la configuration du token ngrok: {e}")


# 3. Tuer les anciens tunnels ngrok au cas où
try:
    ngrok.kill()
    print("Tunnels ngrok précédents (si existants) fermés.")
except:
    pass # Ignorer si pas de tunnel actif

# 4. Créer le tunnel vers le port 8000
try:
    # Tu peux spécifier une région si besoin, ex: region="eu" ou "us"
    # conf.get_default().region = 'eu'
    public_url = ngrok.connect(8000) # Pointe vers le port où Uvicorn écoute
    print("\n" + "="*70)
    print(f"✅ Tunnel Ngrok créé !")
    print(f"✅ Ton API FastAPI est maintenant accessible publiquement à l'adresse : {public_url}")
    print(f"✅ Ouvre la documentation interactive (Swagger UI) : {public_url}/docs")
    print(f"✅ Ou la documentation alternative (ReDoc) : {public_url}/redoc")
    print("="*70 + "\n")
    print("ℹ️ Laisse cette cellule ET la cellule Uvicorn actives pour maintenir l'accès.")

except Exception as e:
    print(f"❌ Erreur lors de la création du tunnel ngrok : {e}")

Vérification/Configuration Ngrok...
Utilisation du token ngrok depuis les secrets Colab.
Tunnels ngrok précédents (si existants) fermés.

✅ Tunnel Ngrok créé !
✅ Ton API FastAPI est maintenant accessible publiquement à l'adresse : NgrokTunnel: "https://2d7f-34-60-205-60.ngrok-free.app" -> "http://localhost:8000"
✅ Ouvre la documentation interactive (Swagger UI) : NgrokTunnel: "https://2d7f-34-60-205-60.ngrok-free.app" -> "http://localhost:8000"/docs
✅ Ou la documentation alternative (ReDoc) : NgrokTunnel: "https://2d7f-34-60-205-60.ngrok-free.app" -> "http://localhost:8000"/redoc

ℹ️ Laisse cette cellule ET la cellule Uvicorn actives pour maintenir l'accès.


In [4]:
!pip install pytest httpx -q
print("pytest et httpx installés pour cette session Colab.")

pytest et httpx installés pour cette session Colab.


In [6]:
# Naviguer au bon endroit (dossier P7)
%cd /content/drive/MyDrive/Colab Notebooks/OC/P7/

# Lancer pytest sur le dossier des tests
!pytest api/tests/

/content/drive/MyDrive/Colab Notebooks/OC/P7
============================= test session starts ==============================
platform linux -- Python 3.11.12, pytest-8.3.5, pluggy-1.5.0
rootdir: /content/drive/MyDrive/Colab Notebooks/OC/P7
plugins: typeguard-4.4.2, langsmith-0.3.38, anyio-4.9.0
collected 9 items                                                              

api/tests/test_main.py .........                                         [100%]

============================== 9 passed in 16.89s ==============================


In [7]:
# Chemin source (ton dossier API sur Drive)
source_api_dir = "/content/drive/MyDrive/Colab Notebooks/OC/P7/api/"
# Chemin destination (dans le dépôt cloné)
dest_repo_dir = "/content/OC/"

print(f"Copie de {source_api_dir} vers {dest_repo_dir}...")
# Utiliser -r pour copier le dossier récursivement, -v pour voir les fichiers copiés
!cp -rv "{source_api_dir}" "{dest_repo_dir}"
print("\nCopie terminée. Vérifiez la présence du dossier 'api' dans /content/OC/api")

# Vérifier le contenu copié (optionnel)
!ls -l /content/OC/api/
!ls -l /content/OC/api/tests/

Copie de /content/drive/MyDrive/Colab Notebooks/OC/P7/api/ vers /content/OC/...
'/content/drive/MyDrive/Colab Notebooks/OC/P7/api/' -> '/content/OC/'
'/content/drive/MyDrive/Colab Notebooks/OC/P7/api/Copie de final_lstm_full_data.keras' -> '/content/OC/Copie de final_lstm_full_data.keras'
'/content/drive/MyDrive/Colab Notebooks/OC/P7/api/Copie de preprocessing_config.json' -> '/content/OC/Copie de preprocessing_config.json'
'/content/drive/MyDrive/Colab Notebooks/OC/P7/api/Copie de tokenizer_full.pkl' -> '/content/OC/Copie de tokenizer_full.pkl'
'/content/drive/MyDrive/Colab Notebooks/OC/P7/api/__pycache__' -> '/content/OC/__pycache__'
'/content/drive/MyDrive/Colab Notebooks/OC/P7/api/__pycache__/main.cpython-311.pyc' -> '/content/OC/__pycache__/main.cpython-311.pyc'
'/content/drive/MyDrive/Colab Notebooks/OC/P7/api/main.py' -> '/content/OC/main.py'
'/content/drive/MyDrive/Colab Notebooks/OC/P7/api/Test_API.ipynb' -> '/content/OC/Test_API.ipynb'
'/content/drive/MyDrive/Colab Notebooks/